In [1]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,OneHotEncoder
import pandas as pd
import tensorflow as tf
import os
from tensorflow.keras.callbacks import ModelCheckpoint

#  Import and read the charity_data.csv.
import pandas as pd 
application_df = pd.read_csv("Resources/charity_data.csv")
application_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


### Attempt 1: Decreasing Columns, Changing bin amounts , and epoch = 40

In [2]:
# Drop the non-beneficial ID columns, 'EIN','NAME''ORGANIZATION','CLASSIFICATION'.
attempt1_df= application_df.drop(['EIN','NAME','ORGANIZATION','CLASSIFICATION'],axis=1)

In [3]:
# Determine the number of unique values in each column.
attempt1_df.nunique()

APPLICATION_TYPE            17
AFFILIATION                  6
USE_CASE                     5
STATUS                       2
INCOME_AMT                   9
SPECIAL_CONSIDERATIONS       2
ASK_AMT                   8747
IS_SUCCESSFUL                2
dtype: int64

In [4]:
# Look at APPLICATION_TYPE value counts for binning
app_type_counts = attempt1_df.APPLICATION_TYPE.value_counts()
app_type_counts

T3     27037
T4      1542
T6      1216
T5      1173
T19     1065
T8       737
T7       725
T10      528
T9       156
T13       66
T12       27
T2        16
T25        3
T14        3
T29        2
T15        2
T17        1
Name: APPLICATION_TYPE, dtype: int64

In [5]:
# Determine which values to replace if counts are less than ...?
replace_application =  list(app_type_counts[app_type_counts<700].index)

# Replace in dataframe
for app in replace_application:
    attempt1_df.APPLICATION_TYPE = attempt1_df.APPLICATION_TYPE.replace(app,"Other")
    
# Check to make sure binning was successful
attempt1_df.APPLICATION_TYPE.value_counts()

T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
Other      804
T8         737
T7         725
Name: APPLICATION_TYPE, dtype: int64

In [6]:
# Generate our categorical variable lists
application_cat = attempt1_df.dtypes[application_df.dtypes=="object"].index.tolist()

In [7]:
# Create a OneHotEncoder instance
enc = OneHotEncoder(sparse=False)

# Fit and transform the OneHotEncoder using the categorical variable list
encode_df = pd.DataFrame(enc.fit_transform(attempt1_df[application_cat]))

# Add the encoded variable names to the dataframe
encode_df.columns = enc.get_feature_names(application_cat)
encode_df.head()

C:\Users\HP\anaconda3\envs\PythonData\envs\mlenv\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


,APPLICATION_TYPE_Other,APPLICATION_TYPE_T19,APPLICATION_TYPE_T3,APPLICATION_TYPE_T4,APPLICATION_TYPE_T5,APPLICATION_TYPE_T6,APPLICATION_TYPE_T7,APPLICATION_TYPE_T8,AFFILIATION_CompanySponsored,AFFILIATION_Family/Parent,...,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M,SPECIAL_CONSIDERATIONS_N,SPECIAL_CONSIDERATIONS_Y
0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
3,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
4,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


In [8]:
# Merge one-hot encoded features and drop the originals
attempt1_df = attempt1_df.merge(encode_df,left_index=True,right_index=True)
attempt1_df = attempt1_df.drop(application_cat,1)
attempt1_df.head()

C:\Users\HP\AppData\Roaming\Python\Python37\site-packages\ipykernel_launcher.py:3: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  This is separate from the ipykernel package so we can avoid doing imports until


,STATUS,ASK_AMT,IS_SUCCESSFUL,APPLICATION_TYPE_Other,APPLICATION_TYPE_T19,APPLICATION_TYPE_T3,APPLICATION_TYPE_T4,APPLICATION_TYPE_T5,APPLICATION_TYPE_T6,APPLICATION_TYPE_T7,...,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M,SPECIAL_CONSIDERATIONS_N,SPECIAL_CONSIDERATIONS_Y
0,1,5000,1,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,1,108590,1,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,1,5000,0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
3,1,6692,1,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
4,1,142590,1,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


In [9]:
# Split our preprocessed data into our features and target arrays
y = attempt1_df["IS_SUCCESSFUL"].values
X = attempt1_df.drop(["IS_SUCCESSFUL"],axis=1).values
# Split the preprocessed data into a training and testing dataset
X_train,X_test, y_train, y_test = train_test_split(X,y,random_state=78)

In [10]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [11]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
number_input_features = len(X_train[0])
hidden_nodes_layer1 = 80
hidden_nodes_layer2 = 30

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(
    tf.keras.layers.Dense(units=hidden_nodes_layer1,input_dim=number_input_features,activation="relu")
)

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 80)                2640      
                                                                 
 dense_1 (Dense)             (None, 30)                2430      
                                                                 
 dense_2 (Dense)             (None, 1)                 31        
                                                                 
Total params: 5,101
Trainable params: 5,101
Non-trainable params: 0
_________________________________________________________________


In [12]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [13]:

# Define the checkpoint path and filenames
os.makedirs("checkpoints/",exist_ok=True)
checkpoint_path = "checkpoints/weights.{epoch:02d}.hdf5"

# Create a callback that saves the model's weights every epoch
cp_callback = ModelCheckpoint(
    filepath=checkpoint_path,
    verbose=1,
    save_weights_only=True,
    save_freq=5)

In [14]:
# Train the model
fit_model = nn.fit(X_train,y_train,epochs=98,callbacks=[cp_callback])

Epoch 1/98
  1/804 [..............................] - ETA: 8:13 - loss: 18550.1816 - accuracy: 0.3750
Epoch 1: saving model to checkpoints\weights.01.hdf5

Epoch 1: saving model to checkpoints\weights.01.hdf5
 10/804 [..............................] - ETA: 4s - loss: 43460.6016 - accuracy: 0.4719  
Epoch 1: saving model to checkpoints\weights.01.hdf5

Epoch 1: saving model to checkpoints\weights.01.hdf5
 23/804 [..............................] - ETA: 3s - loss: 22392.6875 - accuracy: 0.4688
Epoch 1: saving model to checkpoints\weights.01.hdf5

Epoch 1: saving model to checkpoints\weights.01.hdf5

Epoch 1: saving model to checkpoints\weights.01.hdf5
 35/804 [>.............................] - ETA: 3s - loss: 20999.4004 - accuracy: 0.4670
Epoch 1: saving model to checkpoints\weights.01.hdf5

Epoch 1: saving model to checkpoints\weights.01.hdf5
 49/804 [>.............................] - ETA: 3s - loss: 17282.6641 - accuracy: 0.4713
Epoch 1: saving model to checkpoints\weights.01.hdf5

Epoc


Epoch 1: saving model to checkpoints\weights.01.hdf5

Epoch 1: saving model to checkpoints\weights.01.hdf5
445/804 [===============>..............] - ETA: 1s - loss: 24364.2070 - accuracy: 0.4822
Epoch 1: saving model to checkpoints\weights.01.hdf5

Epoch 1: saving model to checkpoints\weights.01.hdf5
458/804 [================>.............] - ETA: 1s - loss: 24105.3848 - accuracy: 0.4821
Epoch 1: saving model to checkpoints\weights.01.hdf5

Epoch 1: saving model to checkpoints\weights.01.hdf5

Epoch 1: saving model to checkpoints\weights.01.hdf5
470/804 [================>.............] - ETA: 1s - loss: 38651.3867 - accuracy: 0.4827
Epoch 1: saving model to checkpoints\weights.01.hdf5

Epoch 1: saving model to checkpoints\weights.01.hdf5
480/804 [================>.............] - ETA: 1s - loss: 39621.7969 - accuracy: 0.4835
Epoch 1: saving model to checkpoints\weights.01.hdf5

Epoch 1: saving model to checkpoints\weights.01.hdf5
491/804 [=================>............] - ETA: 1s - l

 71/804 [=>............................] - ETA: 3s - loss: 27172.0391 - accuracy: 0.5009
Epoch 2: saving model to checkpoints\weights.02.hdf5

Epoch 2: saving model to checkpoints\weights.02.hdf5
 82/804 [==>...........................] - ETA: 3s - loss: 23961.2891 - accuracy: 0.5004
Epoch 2: saving model to checkpoints\weights.02.hdf5

Epoch 2: saving model to checkpoints\weights.02.hdf5
 91/804 [==>...........................] - ETA: 3s - loss: 22625.6660 - accuracy: 0.4945
Epoch 2: saving model to checkpoints\weights.02.hdf5

Epoch 2: saving model to checkpoints\weights.02.hdf5
102/804 [==>...........................] - ETA: 3s - loss: 21643.6523 - accuracy: 0.4920
Epoch 2: saving model to checkpoints\weights.02.hdf5

Epoch 2: saving model to checkpoints\weights.02.hdf5
114/804 [===>..........................] - ETA: 3s - loss: 20663.7715 - accuracy: 0.4852
Epoch 2: saving model to checkpoints\weights.02.hdf5

Epoch 2: saving model to checkpoints\weights.02.hdf5

Epoch 2: saving mod


Epoch 2: saving model to checkpoints\weights.02.hdf5
524/804 [==================>...........] - ETA: 1s - loss: 27736.4121 - accuracy: 0.4865
Epoch 2: saving model to checkpoints\weights.02.hdf5

Epoch 2: saving model to checkpoints\weights.02.hdf5

Epoch 2: saving model to checkpoints\weights.02.hdf5
536/804 [===================>..........] - ETA: 1s - loss: 27184.1055 - accuracy: 0.4859
Epoch 2: saving model to checkpoints\weights.02.hdf5

Epoch 2: saving model to checkpoints\weights.02.hdf5
548/804 [===================>..........] - ETA: 1s - loss: 26789.4043 - accuracy: 0.4856
Epoch 2: saving model to checkpoints\weights.02.hdf5

Epoch 2: saving model to checkpoints\weights.02.hdf5

Epoch 2: saving model to checkpoints\weights.02.hdf5
561/804 [===================>..........] - ETA: 1s - loss: 26826.2207 - accuracy: 0.4849
Epoch 2: saving model to checkpoints\weights.02.hdf5

Epoch 2: saving model to checkpoints\weights.02.hdf5
573/804 [====================>.........] - ETA: 1s - l


Epoch 3: saving model to checkpoints\weights.03.hdf5

Epoch 3: saving model to checkpoints\weights.03.hdf5
162/804 [=====>........................] - ETA: 3s - loss: 209568.3438 - accuracy: 0.5143
Epoch 3: saving model to checkpoints\weights.03.hdf5

Epoch 3: saving model to checkpoints\weights.03.hdf5
173/804 [=====>........................] - ETA: 3s - loss: 197681.4844 - accuracy: 0.5094
Epoch 3: saving model to checkpoints\weights.03.hdf5

Epoch 3: saving model to checkpoints\weights.03.hdf5
184/804 [=====>........................] - ETA: 2s - loss: 185931.5469 - accuracy: 0.5080
Epoch 3: saving model to checkpoints\weights.03.hdf5

Epoch 3: saving model to checkpoints\weights.03.hdf5
196/804 [======>.......................] - ETA: 2s - loss: 183089.6094 - accuracy: 0.5067
Epoch 3: saving model to checkpoints\weights.03.hdf5

Epoch 3: saving model to checkpoints\weights.03.hdf5

Epoch 3: saving model to checkpoints\weights.03.hdf5
207/804 [======>.......................] - ETA: 2s

...

631/804 [======================>.......] - ETA: 0s - loss: 0.6911 - accuracy: 0.5327
Epoch 96: saving model to checkpoints\weights.96.hdf5

Epoch 96: saving model to checkpoints\weights.96.hdf5
640/804 [======================>.......] - ETA: 0s - loss: 0.6912 - accuracy: 0.5318
Epoch 96: saving model to checkpoints\weights.96.hdf5

Epoch 96: saving model to checkpoints\weights.96.hdf5
650/804 [=======================>......] - ETA: 0s - loss: 0.6912 - accuracy: 0.5318
Epoch 96: saving model to checkpoints\weights.96.hdf5

Epoch 96: saving model to checkpoints\weights.96.hdf5
660/804 [=======================>......] - ETA: 0s - loss: 0.6912 - accuracy: 0.5320
Epoch 96: saving model to checkpoints\weights.96.hdf5

Epoch 96: saving model to checkpoints\weights.96.hdf5
671/804 [========================>.....] - ETA: 0s - loss: 0.6911 - accuracy: 0.5325
Epoch 96: saving model to checkpoints\weights.96.hdf5

Epoch 96: saving model to checkpoints\weights.96.hdf5
682/804 [=====================

256/804 [========>.....................] - ETA: 2s - loss: 0.6913 - accuracy: 0.5312
Epoch 97: saving model to checkpoints\weights.97.hdf5

Epoch 97: saving model to checkpoints\weights.97.hdf5
268/804 [=========>....................] - ETA: 2s - loss: 0.6911 - accuracy: 0.5323
Epoch 97: saving model to checkpoints\weights.97.hdf5

Epoch 97: saving model to checkpoints\weights.97.hdf5
277/804 [=========>....................] - ETA: 2s - loss: 0.6912 - accuracy: 0.5320
Epoch 97: saving model to checkpoints\weights.97.hdf5

Epoch 97: saving model to checkpoints\weights.97.hdf5
287/804 [=========>....................] - ETA: 2s - loss: 0.6913 - accuracy: 0.5312
Epoch 97: saving model to checkpoints\weights.97.hdf5

Epoch 97: saving model to checkpoints\weights.97.hdf5
298/804 [==========>...................] - ETA: 2s - loss: 0.6914 - accuracy: 0.5304
Epoch 97: saving model to checkpoints\weights.97.hdf5

Epoch 97: saving model to checkpoints\weights.97.hdf5
308/804 [==========>..........

700/804 [=========================>....] - ETA: 0s - loss: 0.6914 - accuracy: 0.5302
Epoch 97: saving model to checkpoints\weights.97.hdf5

Epoch 97: saving model to checkpoints\weights.97.hdf5

Epoch 97: saving model to checkpoints\weights.97.hdf5
711/804 [=========================>....] - ETA: 0s - loss: 0.6913 - accuracy: 0.5306
Epoch 97: saving model to checkpoints\weights.97.hdf5

Epoch 97: saving model to checkpoints\weights.97.hdf5
723/804 [=========================>....] - ETA: 0s - loss: 0.6912 - accuracy: 0.5314
Epoch 97: saving model to checkpoints\weights.97.hdf5

Epoch 97: saving model to checkpoints\weights.97.hdf5
735/804 [==========================>...] - ETA: 0s - loss: 0.6913 - accuracy: 0.5310
Epoch 97: saving model to checkpoints\weights.97.hdf5

Epoch 97: saving model to checkpoints\weights.97.hdf5

Epoch 97: saving model to checkpoints\weights.97.hdf5
746/804 [==========================>...] - ETA: 0s - loss: 0.6913 - accuracy: 0.5308
Epoch 97: saving model to che

339/804 [===========>..................] - ETA: 2s - loss: 0.6912 - accuracy: 0.5319
Epoch 98: saving model to checkpoints\weights.98.hdf5

Epoch 98: saving model to checkpoints\weights.98.hdf5
350/804 [============>.................] - ETA: 2s - loss: 0.6910 - accuracy: 0.5329
Epoch 98: saving model to checkpoints\weights.98.hdf5

Epoch 98: saving model to checkpoints\weights.98.hdf5

Epoch 98: saving model to checkpoints\weights.98.hdf5
362/804 [============>.................] - ETA: 2s - loss: 0.6910 - accuracy: 0.5330
Epoch 98: saving model to checkpoints\weights.98.hdf5

Epoch 98: saving model to checkpoints\weights.98.hdf5
375/804 [============>.................] - ETA: 2s - loss: 0.6910 - accuracy: 0.5332
Epoch 98: saving model to checkpoints\weights.98.hdf5

Epoch 98: saving model to checkpoints\weights.98.hdf5
386/804 [=============>................] - ETA: 1s - loss: 0.6910 - accuracy: 0.5334
Epoch 98: saving model to checkpoints\weights.98.hdf5

Epoch 98: saving model to che


Epoch 98: saving model to checkpoints\weights.98.hdf5
794/804 [============================>.] - ETA: 0s - loss: 0.6912 - accuracy: 0.5322
Epoch 98: saving model to checkpoints\weights.98.hdf5

Epoch 98: saving model to checkpoints\weights.98.hdf5
804/804 [==============================] - 4s 5ms/step - loss: 0.6912 - accuracy: 0.5321


In [15]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

#Accuracy without callback in fit.model statement was about 0.71

268/268 - 0s - loss: 1.0432 - accuracy: 0.5334 - 433ms/epoch - 2ms/step
Loss: 1.0432437658309937, Accuracy: 0.5334110856056213


In [16]:
#Export model to HDF5 file
nn.save("AlphabetSoupCharity_Optimization.h5")

### Attempt 2: Adding additional layers, increasing neurons and changing epoch to 70

In [17]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
number_input_features = len(X_train[0])

nn_new2 = tf.keras.models.Sequential()

# First hidden layer
nn_new2.add(tf.keras.layers.Dense(units=90,input_dim=number_input_features,activation="relu"))

# Second hidden layer
nn_new2.add(tf.keras.layers.Dense(units=70, activation="relu"))

#Third hidden layer
nn_new2.add(tf.keras.layers.Dense(units=50,activation='relu'))

# Output layer
nn_new2.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn_new2.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_3 (Dense)             (None, 90)                2970      
                                                                 
 dense_4 (Dense)             (None, 70)                6370      
                                                                 
 dense_5 (Dense)             (None, 50)                3550      
                                                                 
 dense_6 (Dense)             (None, 1)                 51        
                                                                 
Total params: 12,941
Trainable params: 12,941
Non-trainable params: 0
_________________________________________________________________


In [18]:
# Compile the model
nn_new2.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [19]:
# Train the model
fit_model = nn_new2.fit(X_train,y_train,epochs=98)

Epoch 1/98
804/804 [==============================] - 2s 2ms/step - loss: 28889.2051 - accuracy: 0.4872
Epoch 2/98
804/804 [==============================] - 1s 2ms/step - loss: 5558.1025 - accuracy: 0.4958
Epoch 3/98
804/804 [==============================] - 1s 2ms/step - loss: 1197.6770 - accuracy: 0.4963
Epoch 4/98
804/804 [==============================] - 1s 2ms/step - loss: 63.0711 - accuracy: 0.5219
Epoch 5/98
804/804 [==============================] - 1s 2ms/step - loss: 0.6928 - accuracy: 0.5321
Epoch 6/98
804/804 [==============================] - 1s 2ms/step - loss: 0.6911 - accuracy: 0.5321
Epoch 7/98
804/804 [==============================] - 1s 2ms/step - loss: 0.6912 - accuracy: 0.5321
Epoch 8/98
804/804 [==============================] - 1s 2ms/step - loss: 0.6912 - accuracy: 0.5321
Epoch 9/98
804/804 [==============================] - 1s 2ms/step - loss: 0.6912 - accuracy: 0.5321
Epoch 10/98
804/804 [==============================] - 1s 2ms/step - loss: 0.6912 - accur

804/804 [==============================] - 1s 2ms/step - loss: 0.6912 - accuracy: 0.5321
Epoch 83/98
804/804 [==============================] - 1s 2ms/step - loss: 0.6912 - accuracy: 0.5321
Epoch 84/98
804/804 [==============================] - 1s 2ms/step - loss: 0.6912 - accuracy: 0.5321
Epoch 85/98
804/804 [==============================] - 1s 2ms/step - loss: 0.6912 - accuracy: 0.5321
Epoch 86/98
804/804 [==============================] - 1s 2ms/step - loss: 0.6912 - accuracy: 0.5321
Epoch 87/98
804/804 [==============================] - 1s 2ms/step - loss: 0.6912 - accuracy: 0.5321
Epoch 88/98
804/804 [==============================] - 1s 2ms/step - loss: 0.6912 - accuracy: 0.5321
Epoch 89/98
804/804 [==============================] - 1s 2ms/step - loss: 0.6912 - accuracy: 0.5321
Epoch 90/98
804/804 [==============================] - 1s 2ms/step - loss: 0.6912 - accuracy: 0.5321
Epoch 91/98
804/804 [==============================] - 1s 2ms/step - loss: 0.6912 - accuracy: 0.5321
Ep

In [20]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn_new2.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.7187 - accuracy: 0.5331 - 420ms/epoch - 2ms/step
Loss: 0.7187100052833557, Accuracy: 0.5330612063407898


### Attempt 3: Adding hidden layers, and increasing neurons, epoch=95 

In [21]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
number_input_features = len(X_train[0])

nn_new1 = tf.keras.models.Sequential()

# First hidden layer
nn_new1.add(tf.keras.layers.Dense(units=100,input_dim=number_input_features,activation="relu"))

# Second hidden layer
nn_new1.add(tf.keras.layers.Dense(units=80, activation="relu"))

#Third hidden layer
nn_new1.add(tf.keras.layers.Dense(units=50,activation='relu'))

#Fourth hidden layer
nn_new1.add(tf.keras.layers.Dense(units=30,activation='linear'))

# Output layer
nn_new1.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn_new1.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_7 (Dense)             (None, 100)               3300      
                                                                 
 dense_8 (Dense)             (None, 80)                8080      
                                                                 
 dense_9 (Dense)             (None, 50)                4050      
                                                                 
 dense_10 (Dense)            (None, 30)                1530      
                                                                 
 dense_11 (Dense)            (None, 1)                 31        
                                                                 
Total params: 16,991
Trainable params: 16,991
Non-trainable params: 0
_________________________________________________________________


In [22]:
# Compile the model
nn_new1.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [23]:
# Train the model
fit_model = nn_new1.fit(X_train,y_train,epochs=95)

Epoch 1/95
804/804 [==============================] - 2s 2ms/step - loss: 103203.9688 - accuracy: 0.4950
Epoch 2/95
804/804 [==============================] - 1s 2ms/step - loss: 39990.4023 - accuracy: 0.4995
Epoch 3/95
804/804 [==============================] - 1s 2ms/step - loss: 57528.1289 - accuracy: 0.4964
Epoch 4/95
804/804 [==============================] - 1s 1ms/step - loss: 3767.0559 - accuracy: 0.5089
Epoch 5/95
804/804 [==============================] - 1s 1ms/step - loss: 15009.7432 - accuracy: 0.4940
Epoch 6/95
804/804 [==============================] - 1s 1ms/step - loss: 3479.3406 - accuracy: 0.4985
Epoch 7/95
804/804 [==============================] - 1s 1ms/step - loss: 861.7783 - accuracy: 0.5028
Epoch 8/95
804/804 [==============================] - 1s 2ms/step - loss: 0.7336 - accuracy: 0.5321
Epoch 9/95
804/804 [==============================] - 1s 1ms/step - loss: 0.6913 - accuracy: 0.5321
Epoch 10/95
804/804 [==============================] - 1s 2ms/step - loss: 

Epoch 82/95
804/804 [==============================] - 1s 2ms/step - loss: 0.6911 - accuracy: 0.5321
Epoch 83/95
804/804 [==============================] - 1s 2ms/step - loss: 0.6912 - accuracy: 0.5321
Epoch 84/95
804/804 [==============================] - 1s 2ms/step - loss: 0.6911 - accuracy: 0.5321
Epoch 85/95
804/804 [==============================] - 1s 2ms/step - loss: 0.6911 - accuracy: 0.5321
Epoch 86/95
804/804 [==============================] - 1s 2ms/step - loss: 0.6911 - accuracy: 0.5321
Epoch 87/95
804/804 [==============================] - 1s 2ms/step - loss: 0.6911 - accuracy: 0.5321
Epoch 88/95
804/804 [==============================] - 1s 2ms/step - loss: 0.6911 - accuracy: 0.5321
Epoch 89/95
804/804 [==============================] - 1s 2ms/step - loss: 0.6912 - accuracy: 0.5321
Epoch 90/95
804/804 [==============================] - 1s 2ms/step - loss: 0.6911 - accuracy: 0.5321
Epoch 91/95
804/804 [==============================] - 1s 2ms/step - loss: 0.6911 - accurac

In [24]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn_new1.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.6901 - accuracy: 0.5347 - 470ms/epoch - 2ms/step
Loss: 0.6901078820228577, Accuracy: 0.5346938967704773
